In [4]:
import pandas as pd

# Đọc dữ liệu từ file all_labeled.json
df = pd.read_json('all_labeled.json')

# Hiển thị thông tin tổng quan về dataframe
print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nInfo:")
print(df.info())
print("\nMissing values per column:")
print(df.isnull().sum())
print("\nSample data:")
print(df.head())

Shape: (2262, 19)

Columns: ['id', 'annotations', 'file_upload', 'drafts', 'predictions', 'data', 'meta', 'created_at', 'updated_at', 'inner_id', 'total_annotations', 'cancelled_annotations', 'total_predictions', 'comment_count', 'unresolved_comment_count', 'last_comment_updated_at', 'project', 'updated_by', 'comment_authors']

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2262 entries, 0 to 2261
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   id                        2262 non-null   int64              
 1   annotations               2262 non-null   object             
 2   file_upload               2262 non-null   object             
 3   drafts                    2262 non-null   object             
 4   predictions               2262 non-null   object             
 5   data                      2262 non-null   object             
 6   meta            